## This code is a programming tutorial for the paper " Interpretable Multi-Task XGBoost Model: An Enhanced Method to Estimate Reservoir Parameters from Logging Data"

## Author: Keran Li

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import xgboost as xgb
import pandas as pd
import numpy as np

In [ ]:
# import data
data = pd.read_csv("your_file_path")

In [ ]:
# define label_processed
def label_p(x1):
    matrix = np.array([100,10,1,0.1])
    bais_estimated = np.linalg.norm(matrix)
    return 1/(1+np.exp(np.linalg.norm(x)+bais_estimated))

### After you transfer the lithological labels (one-hot), the litho-classification becomes a regression task. We could use a XGBRegressor to complete the 1st-stage feature extraction 

In [ ]:
# Use the processed label to train
class first_stage_XGB(nn.Module):
    def init(self, input_dim, output_dim, learning_rate, num_epochs):
        super(XGBRegressor, self).init()
        self.input_dim = input_dim
        self.output_dim = output_dim
        self.learning_rate = learning_rate
        self.num_epochs = num_epochs
        self.linear = nn.Linear(self.input_dim, self.output_dim)
        self.sigmoid = nn.Sigmoid()
        self.criterion = nn.MSELoss()
        self.optimizer = optim.Adam(self.parameters(), lr=self.learning_rate)

    # define foward
    def forward(self, X, y):
    self.model.fit(X, y)
    return self.model.predict(X)

    # model training
    def train(self, x, y):
        for epoch in range(self.num_epochs):
            # Forward pass
            outputs = self.forward(x)
            # Calculate loss
            loss = self.criterion(outputs, y)
            # Backward pass
            self.optimizer.zero_grad()
            loss.backward()
            self.optimizer.step()
            # Print loss
            print('Epoch [{}/{}], Loss: {:.4f}'.format(epoch+1, self.num_epochs, loss.item()))
        
xgb_regressor = first_stage_XGB(input_dim="Number of logging data types", output_dim=1, ="The_learning_rate_you_want", num_epochs="The_num_epochs_you_want")
xgb_regressor.train("Logging data", "Previous processed label")

In [ ]:
# define inverse_solution
def inve_sol(x2):
    return np.log(1/x2-1)

In [ ]:
# define task_calssification
def task_class(x3):
    x = inve_sol(x3)
    return np.argmax(np.exp(x)/np.sum(np.exp(x)))

In [ ]:
# define choose
def choose(x4):
    p = np.array([0,0,0,1])
    w = np.array([0,0,1,0])
    m = np.array([0,1,0,0])
    mm = np.array([1,0,0,0])
    def ed(m1,m2):
        return np.sqrt(np.sum((m1-m2)**2))
    for i in range(4):
        lith_class = np.array([ed(x4,p),ed(x4,w),ed(x4,m),ed(x4,mm)])
    return  lith_class

In [ ]:
# define decoder
def decoder(x5):
    if task_class(x5)==0:
        return choose(inve_sol(x5)/100-1)
    if task_class(x5)==1:
        return inve_sol(x5)/10-1
    if task_class(x5)==2:
        return inve_sol(x5)-1
    if task_class(x5)==3:
        return inve_sol(x5)*10-1 

In [ ]:
# define gate
def gate(x5,label):
    return np.sum(x5,label)*0.5

### After decoder and gate, four XGBoost models are adopted for specific tasks

In [ ]:

class second_stage_multi_XGB(nn.Module):
    def init(self, input_dim_clf, output_dim_clf,input_dim_reg,
             output_dim_reg, learning_rate, num_epochs):
        super(second_stage_multi_XGB, self).init()
        self.input_dim_clf = input_dim_clf
        self.output_dim_clf = output_dim_clf
        self.input_dim_reg = input_dim_reg
        self.output_dim_reg = output_dim_reg
        self.learning_rate = learning_rate
        self.num_epochs = num_epochs
        self.linear = nn.Linear(self.input_dim_cls, self.output_dim_cls)
        self.sigmoid = nn.Sigmoid()
        self.criterion_cls = nn.MSELoss()
        self.criterion_reg = nn.CrossEntropyLoss()

        self.xgb_clf = xgb.XGBClassifier()
        self.xgb_reg1 = xgb.XGBRegressor()
        self.xgb_reg2 = xgb.XGBRegressor()
        self.xgb_reg3 = xgb.XGBRegressor()        
        
        self.optimizer = optim.Adam(self.parameters(), lr=self.learning_rate)
        
    def forward(self, x):
        # Classification task
        clf_out = self.xgb_clf(x)
        # Regression tasks
        reg1_out = self.xgb_reg1(x)
        reg2_out = self.xgb_reg2(x)
        reg3_out = self.xgb_reg3(x)
        return clf_out, reg1_out, reg2_out, reg3_out
    
    def multi_task_loss(clf_out,reg1_out, reg2_out, reg3_out,clf_target,reg1_target, reg2_target, reg3_target):
        clf_loss = F.cross_entropy(clf_out, clf_target)
        reg1_loss = F.mse_loss(reg1_out, reg1_target)
        reg2_loss = F.mse_loss(reg2_out, reg2_target)
        reg3_loss = F.mse_loss(reg3_out, reg3_target)
        return clf_loss + reg1_loss + reg2_loss + reg3_loss
    
    def train(self, x, y):
        for epoch in range(self.num_epochs):            
            # Forward pass
            outputs = self.forward(x)
            # Calculate loss
            loss = loss = multi_task_loss(clf_out, reg1_out, reg2_out, reg3_out, clf_target, reg1_target, reg2_target, reg3_target)
            # Backward pass
            self.optimizer.zero_grad()
            loss.backward()
            self.optimizer.step()
            # Print loss
            print('Epoch [{}/{}], Loss: {:.4f}'.format(epoch+1, self.num_epochs, loss.item()))

MT_XGB = second_stage_multi_XGB(input_dim_cls="Number of logging data types", output_dim_cls=4,input_dim_reg="Number of logging data types",
             output_dim_reg=1, learning_rate="The_learning_rate_you_want", num_epochs="The_num_epochs_you_want")
MT_XGB.train("Logging data", "Label")            